In [2]:
import pickle
import pandas as pd
from sklearn import preprocessing
import re
import scipy
import numpy as np
import glob
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
from gensim.models.doc2vec import TaggedDocument

2022-05-13 22:50:04.754975: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [31]:
class UsePretrainedVectClf:
    def __init__(self, pathToTokenizer, pathToClfModel, pathToWord2VecPretrained, path_to_PreprocessedDataset, \
                 path_to_nonPreprocessedDataset, include_rules = False):
        self.path_to_nonPreprocessedDataset = path_to_nonPreprocessedDataset
        self.path_to_PreprocessedDataset = path_to_PreprocessedDataset
        self.include_rules = include_rules
        self.pathToTokenizer = pathToTokenizer
        self.pathToClfModel = pathToClfModel
        self.pathToWord2VecPretrained = pathToWord2VecPretrained
    
    def traditional_ChangeVectPath(self, pathToTokenizer):
        self.pathToTokenizer = pathToTokenizer   
        
    def traditional_ChangeClfModelPath(self, pathToClfModel):
        self.pathToClfModel = pathToClfModel     
        
    def traditional_loadVect(self):
        Vect = pickle.load(open(self.pathToTokenizer, 'rb'))
        return Vect
    
    def apply_gen_rules_features_ngrams(self, X, X_train_WoR_dtm, X_train_WoR_Features):
        X_Rules_dtm, features_Rules = self.gen_rules_features_ngrams(X)
        X_train_WR_dtm, combined_features = self.concat_sparse_matrices_h(X_train_WoR_dtm, X_Rules_dtm,
                                                                          X_train_WoR_Features, features_Rules)
        return X_train_WR_dtm, combined_features


    def gen_rules_features_ngrams(self, X_data_series):  # , X_data_dtm, features_arg):
        '''sparse matrix and series matrices should be converted to dataframe for applying rules and treating
        it as features...
        I wrote two functions i.e.,sparse_matrix_to_DataFrame() and series_DataFrame()
          for changing datatypes'''
        
        X_data_DF = self.series_to_DataFrame(X_data_series)
        regexes = [
            re.compile(r'\b(I|we)\b.*\b(am|are|will be)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b',
                       re.I | re.M),
            re.compile(r'\b(I\'m)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
            re.compile(r'\b(we\'re)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(bring|give|help|raise|donate|auction)\b',
                       re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(we\'ll)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(ready|prepared)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
            re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(will be)\b.*\b(brought|given|raised|donated|auctioned)\b', re.I | re.M),
            re.compile(r'\b\w*\s*\b\?', re.I | re.M),
            re.compile(r'\b(you|u).*(can|could|should|want to)\b', re.I | re.M),
            re.compile(r'\b(can|could|should).*(you|u)\b', re.I | re.M),
            re.compile(r'\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M)

        ]
        temp = pd.DataFrame()
        features_arg = []
        for i, regex in zip(range(len(regexes)), regexes):
            columnName = "RegEx_" + str(i + 1)
            features_arg.append(columnName)
            temp[columnName] = X_data_DF['tweet_text'].apply(lambda text: self.apply_regex_ngrams(text, regex))
        temp_sparse = scipy.sparse.csr_matrix(temp.values)
        return temp_sparse, features_arg
    
    def series_to_DataFrame(self, X_data):
        X_data = X_data.to_frame()
        return X_data
    
    def concat_sparse_matrices_h(self, data_X_dtm, data_Rules_dtm, features_X, features_Rules):
        combined_features = features_X + features_Rules
        concat_sparse = scipy.sparse.hstack([data_X_dtm, data_Rules_dtm], format='csr')
        return concat_sparse, combined_features
    
    def apply_regex_ngrams(self, text, regex):
        match_found = (re.search(regex, text) != None)
        match_found = int(match_found == True)
        return match_found
    
    def traditional_getFeatures(self, Vect):
        df = pd.read_csv(self.path_to_PreprocessedDataset, encoding = "ISO-8859-1")
        X_train_WoR_dtm = Vect.transform(df["tweet_text"])
        X_train_Features = list(Vect.get_feature_names_out())
        if self.include_rules:
            df_notPreprocessed = pd.read_csv(self.path_to_nonPreprocessedDataset,  encoding = "ISO-8859-1")
            X_train_dtm, X_train_Features = self.apply_gen_rules_features_ngrams(df_notPreprocessed["tweet_text"], \
                                                                      X_train_WoR_dtm, X_train_Features)
        else:
            X_train_dtm = X_train_WoR_dtm
        return X_train_dtm.toarray(), Vect

    def traditional_loadClf(self):  
        clf_model = pickle.load(open(self.pathToClfModel, 'rb'))
        return clf_model
    
    def get_predictions(self, clf_model, X):
        return clf_model.predict(X)
    
    def generateNameFromDataSetName(self, list1):
        str1 = ""
        for e in list1:
            if e == "":
                str1 += '.'.join(str(e))
            else:
                if e == "csv":
                    e = "_withPredictions." + e
                    str1 += ''.join(str(e))
                else:
                    str1 += ''.join(str(e))
        return ".." + str1        
    
    def evaluation(self):
        if "WRul" in self.pathToTokenizer:
            if not self.include_rules:
                print("Vectoirzer and appending does not match")
        if "WoRul" in self.pathToTokenizer:
            if self.include_rules:
                print("Vectoirzer and appending does not match")
        if "WRul" in self.pathToClfModel:
            if not self.include_rules:
                print("Vectoirzer and appending does not match")
        if "WoRul" in self.pathToClfModel:
            if self.include_rules:
                print("Vectoirzer and appending does not match")
        if not self.pathToClfModel.split("/")[-1].split("_")[5:-1] == self.pathToTokenizer.split("/")[-1].split("_"):
            print("Vectorizer and classifer does not match.")
            print("Vectorizer:", self.pathToTokenizer)
            print("Classifier:", self.pathToClfModel)
            print("Classifier:", self.pathToTokenizer.split("/")[-1].split("_"), "Vectorizer:" , self.pathToClfModel.split("/")[-1].split("_")[5:-1])

    def generatePredNGramsPretrained(self, path_to_nonPreprocessedDataset, path_to_PreprocessedDataset, pathToVectFolder,\
                                         pathToClfModel, path2word2vecModel, pathToWord2VecPretrained):

        NotProcessedDataWithPredictions = pd.read_csv(obj.path_to_nonPreprocessedDataset, encoding = "ISO-8859-1")
        for vect_filename in glob.glob(pathToVectFolder + "/*"):
            print(40*"+")
            obj.traditional_ChangeVectPath(vect_filename)
            Vect = obj.traditional_loadVect()
            for clfModel_filename in glob.glob(pathToClfFolder + "*"):
                if clfModel_filename.split("/")[-1].split("_")[5:-1] == vect_filename.split("/")[-1].split("_"):
                    obj.traditional_ChangeClfModelPath(clfModel_filename)
                    name_cell = obj.pathToClfModel.split("/")[-1].split("_")[5:]
                    if name_cell[-3] == "WRul":
                        obj.include_rules = True
                    else:
                        obj.include_rules = False
                    # print(obj.pathToTokenizer, obj.pathToClfModel)
                    # print(name_cell)
                    # print(name_cell[-5] + "_" + name_cell[-3]  + "_" + name_cell[-4] + "_" + name_cell[-2] + "_" + name_cell[-1])
                    X, _ = obj.traditional_getFeatures(Vect)
                    clf_model = obj.traditional_loadClf()
                    y_pred = obj.get_predictions(clf_model, X)
                    name_cell = name_cell[-5] + "_" + name_cell[-3]  + "_" + name_cell[-4] + "_" + name_cell[-2] + "_" + name_cell[-1]
                    print(name_cell)
                    NotProcessedDataWithPredictions[name_cell] = y_pred.tolist()
        notProcessedDatasetName = obj.generateNameFromDataSetName(obj.path_to_nonPreprocessedDataset.split("."))
        NotProcessedDataWithPredictions.to_csv(notProcessedDatasetName)
        print(40*"-")

   

In [3]:
class Use_Word2Vec:
    def __init__(self,pathToTokenizer, pathToClfModel, pathToWord2VecPretrained, path_to_PreprocessedDataset, \
                 path_to_nonPreprocessedDataset, include_rules = False):
        self.pathToTokenizer = pathToTokenizer
        self.pathToClfModel = pathToClfModel
        self.include_rules = include_rules
        self.pathToWord2VecPretrained = pathToWord2VecPretrained
        self.path_to_nonPreprocessedDataset = path_to_nonPreprocessedDataset
        self.path_to_PreprocessedDataset = path_to_PreprocessedDataset

    
    def word_vector_avg(self, inv_tokenizer, fun_word_to_vec_map, tw_sequence, size):
        vec = np.zeros(size)
        count = 0
        columnNameList = []
        for i in range(18):
            columnName = "rule" + str(i + 1)
            columnNameList.append(columnName)
        for seq in tw_sequence:
            try:
                if inv_tokenizer[seq] in columnNameList:
                    embedding_vector = np.ndarray(shape=(size,))
                    embedding_vector[:] = 1/size
                else:
                    embedding_vector = fun_word_to_vec_map[inv_tokenizer[seq]]
                if embedding_vector is not None:
                    vec += embedding_vector
                else:
                    vec += np.zeros(size)
                count += 1.
            except KeyError:  # handling the case where the token is not in vocabulary
                continue
        if count != 0:
            vec /= count
        return vec

    
    def concat_sparse_matrices_h(self, data_X_dtm, data_Rules_dtm, features_X, features_Rules):
        combined_features = features_X + features_Rules
        concat_sparse = scipy.sparse.hstack([data_X_dtm, data_Rules_dtm], format='csr')
        return concat_sparse, combined_features
    
    def series_to_DataFrame(self, X_data):
        X_data = X_data.to_frame()
        return X_data
    
    def apply_regex_word2vec(self, text, regex, ruleTokenizerSequenceIndex):
        match_found = (re.search(regex, text) != None)
        if match_found:
            match_found = int(ruleTokenizerSequenceIndex)
        else:
            match_found = 0
        return match_found
    
    def gen_rules_features_word2vec(self, X_data_series, tokenizer):  # , X_data_dtm, features_arg):
        '''sparse matrix and series matrices should be converted to dataframe for applying rules and treating
    it as features...
    I wrote two functions i.e.,sparse_matrix_to_DataFrame() and series_DataFrame()
      for changing datatypes'''
        X_data_DF = self.series_to_DataFrame(X_data_series)
        regexes = [
            re.compile(r'\b(I|we)\b.*\b(am|are|will be)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b',
                       re.I | re.M),
            re.compile(r'\b(I\'m|Im)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
            re.compile(r'\b(we\'re|we are)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(bring|give|help|raise|donate|auction)\b',
                       re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(we\'ll|we will)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(ready|prepared)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
            re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(will be)\b.*\b(brought|given|raised|donated|auctioned)\b', re.I | re.M),
            re.compile(r'\b\w*\s*\b\?', re.I | re.M),
            re.compile(r'\b(you|u).*(can|could|should|want to)\b', re.I | re.M),
            re.compile(r'\b(can|could|should).*(you|u)\b', re.I | re.M),
            re.compile(r'\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M)

        ]
        temp = pd.DataFrame()
        features_arg = []
        for i, regex in zip(range(len(regexes)), regexes):
            # we can also use ruleString()
            columnName = "rule" + str(i + 1)
            features_arg.append(columnName)
            temp[columnName] = X_data_DF['tweet_text'].apply(lambda text: self.apply_regex_word2vec(text, regex, tokenizer.word_index[columnName]))

        temp_sparse = scipy.sparse.csr_matrix(temp.values)

        return temp_sparse, features_arg
    def apply_gen_rules_features_word2vec(self, X, X_train_WoR_dtm, tokenizer, X_train_WoR_Features):
        X_Rules_dtm, features_Rules = self.gen_rules_features_word2vec(X, tokenizer)

        X_train_WR_dtm, combined_features = self.concat_sparse_matrices_h(X_train_WoR_dtm, X_Rules_dtm,
                                                                          X_train_WoR_Features, features_Rules)
        return X_train_WR_dtm, combined_feature

    
    def traditional_ChangeVectPath(self, pathToTokenizer):
        self.pathToTokenizer = pathToTokenizer 
        
    def word2vec_ChangeWord2VecPretrainedPath(self, pathToWord2VecPretrained):
        self.pathToWord2VecPretrained = pathToWord2VecPretrained 

    def word2vec_loadWord2VecPretrained(self):  
        word2vec_model = pickle.load(open(self.pathToWord2VecPretrained, 'rb'))
        return word2vec_model
        
    def traditional_loadVect(self):
        Vect = pickle.load(open(self.pathToTokenizer, 'rb'))
        return Vect
    
    def traditional_loadClf(self):  
        clf_model = pickle.load(open(self.pathToClfModel, 'rb'))
        return clf_model

    def traditional_ChangeClfModelPath(self, pathToClfModel):
        self.pathToClfModel = pathToClfModel  

    def invTokenizer(self, tokenizer):
        inv_tokenizer = {v: k for k, v in tokenizer.word_index.items()}
        return inv_tokenizer

    def word2vec_Vect(self, pathToWord2VecPretrained, df):
        tokenizer = pickle.load(open(self.pathToTokenizer, 'rb'))
        embed_dim = int(self.pathToTokenizer.split("Freq-")[1])
        if "WRul" in self.pathToTokenizer:
            MAX_SEQUENCE_LENGTH = int(self.pathToTokenizer.split("Seq-")[1].split("_")[0]) - 18
        else:
            MAX_SEQUENCE_LENGTH = int(self.pathToTokenizer.split("Seq-")[1].split("_")[0])

        sequences = tokenizer.texts_to_sequences(df['tweet_text'])
        X_train_WoR_dtm = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
        X_train_Features = ["_Not_Apply_"]
        # print(type(X_train_WoR_dtm), X_train_WoR_dtm.shape)
        if self.include_rules:
            df_notPreprocessed = pd.read_csv(self.path_to_nonPreprocessedDataset)
            X_train_dtm, _ = self.apply_gen_rules_features(df_notPreprocessed["tweet_text"], X_train_WoR_dtm, tokenizer, \
                                                      X_train_Features)
        else:
            X_train_dtm = X_train_WoR_dtm
        inv_tokenizer = self.invTokenizer(tokenizer)
        fun_word_to_vec_map = self.load_word2vec_model(pathToWord2VecPretrained)
        vector = []
        for tw_sequence in X_train_dtm:
            vec = self.word_vector_avg(inv_tokenizer, fun_word_to_vec_map, tw_sequence, embed_dim)
            vector.append(vec)
        X_train_dtm = np.array(vector)
        return X_train_dtm
    
    def load_word2vec_model(self, path):
        if "Model" not in path:
            print("You may not loading a model; You may loading a tokenizer.")
        if "glove".lower() in path.lower().split("/")[-1]:
            word_to_vec_map = pickle.load(open(path, 'rb'))
        elif "word2vec".lower() in path.lower().split("/")[-1]:
            word_to_vec_map = pickle.load(open(path, 'rb'))
        elif "google".lower() in path.lower().split("/")[-1]:
            word_to_vec_map = pickle.load(open(path, 'rb'))
        elif "doc2vec".lower() in path.lower().split("/")[-1]:
            word_to_vec_map = pickle.load(open(path, 'rb'))
            
        elif "crisisNLP2vec".lower() in path.lower().split("/")[2]:
             word_to_vec_map = pickle.load(open(path, 'rb'))
        return word_to_vec_map
    
    def get_predictions(self, clf_model, X):
        return clf_model.predict(X)
    
    def generateNameFromDataSetName(self, list1):
        str1 = ""
        for e in list1:
            if e == "":
                str1 += '.'.join(str(e))
            else:
                if e == "csv":
                    e = "_withPredictions." + e
                    str1 += ''.join(str(e))
                else:
                    str1 += ''.join(str(e))
        return ".." + str1

    def generatePredWord2VecPretrained(self, path_to_nonPreprocessedDataset, path_to_PreprocessedDataset, pathToVectFolder,\
                                     pathToClfModel, path2word2vecModel, pathToWord2VecPretrained):
        NotProcessedDataWithPredictions = pd.read_csv(obj.path_to_nonPreprocessedDataset, encoding = "ISO-8859-1")
        df = pd.read_csv(obj.path_to_PreprocessedDataset,  encoding = "ISO-8859-1")
        for vect_filename in glob.glob(pathToVectFolder + "/*"):
            print(40*"+")
            obj.traditional_ChangeVectPath(vect_filename)
            Vect = obj.traditional_loadVect()
            for pretrainedWord2VecModel_filename in glob.glob(pathToWord2VecPretrained + "*"): 
                if pretrainedWord2VecModel_filename.split("/")[-1].replace("Model", "Vect").split("_") == \
                vect_filename.split("/")[-1].split("_"):
                    obj.word2vec_ChangeWord2VecPretrainedPath(pretrainedWord2VecModel_filename)
                    for clfModel_filename in glob.glob(pathToClfFolder + "*"):
                        if clfModel_filename.split("/")[-1].split("_")[6:-1] == pretrainedWord2VecModel_filename.split("/")[-1].replace("Model", "Vect").split("_"):
                            obj.traditional_ChangeClfModelPath(clfModel_filename)
                            name_cell = obj.pathToClfModel.split("/")[-1].split("_")[5:]
                            if name_cell[-3] == "WRul":
                                obj.include_rules = True
                            else:
                                obj.include_rules = False
                            X = obj.word2vec_Vect(pretrainedWord2VecModel_filename, df)
                            clf_model = obj.traditional_loadClf()
                            y_pred = obj.get_predictions(clf_model, X)
                            name_cell = name_cell[-6] + "_" + name_cell[-3]  + "_" + name_cell[-4] + "_" + name_cell[-2] + "_" + name_cell[-1]
                            print(name_cell)
                            NotProcessedDataWithPredictions[name_cell] = y_pred.tolist()
        notProcessedDatasetName = obj.generateNameFromDataSetName(obj.path_to_nonPreprocessedDataset.split("."))
        NotProcessedDataWithPredictions.to_csv(notProcessedDatasetName)
        print(40*"-")
    

pathToClfModel = "" 
path_to_nonPreprocessedDataset = "../datasets/500_random_sample.csv"
path_to_PreprocessedDataset = "../datasets/500_random_sample_processed.csv"
pathToVectFolder = "../word2vec_vect/"
pathToClfFolder = "../word2vec_clfModels/"


path2word2vecModel = 'dummyPath/word2vecModels/word2vec'
pathToWord2VecPretrained = "../word2vec_preTrainedModel/"
obj = Use_Word2Vec(pathToVectFolder, pathToClfModel, pathToWord2VecPretrained, path_to_PreprocessedDataset, \
                   path_to_nonPreprocessedDataset)
obj.generatePredWord2VecPretrained(path_to_nonPreprocessedDataset, path_to_PreprocessedDataset, pathToVectFolder,\
                                 pathToClfModel, path2word2vecModel, pathToWord2VecPretrained)

++++++++++++++++++++++++++++++++++++++++
word2vec_Max-Len-Seq-20_WoRul_Freq-300_DT
word2vec_Max-Len-Seq-20_WoRul_Freq-300_GB
word2vec_Max-Len-Seq-20_WoRul_Freq-300_LR
word2vec_Max-Len-Seq-20_WoRul_Freq-300_MLP
word2vec_Max-Len-Seq-20_WoRul_Freq-300_NB
word2vec_Max-Len-Seq-20_WoRul_Freq-300_RF
word2vec_Max-Len-Seq-20_WoRul_Freq-300_SVM
++++++++++++++++++++++++++++++++++++++++
doc2vec_Max-Len-Seq-20_WoRul_Freq-300_DT
doc2vec_Max-Len-Seq-20_WoRul_Freq-300_GB
doc2vec_Max-Len-Seq-20_WoRul_Freq-300_LR
doc2vec_Max-Len-Seq-20_WoRul_Freq-300_MLP
doc2vec_Max-Len-Seq-20_WoRul_Freq-300_NB
doc2vec_Max-Len-Seq-20_WoRul_Freq-300_RF
doc2vec_Max-Len-Seq-20_WoRul_Freq-300_SVM
++++++++++++++++++++++++++++++++++++++++
glove_Max-Len-Seq-20_WoRul_Freq-200_DT
glove_Max-Len-Seq-20_WoRul_Freq-200_GB
glove_Max-Len-Seq-20_WoRul_Freq-200_LR
glove_Max-Len-Seq-20_WoRul_Freq-200_MLP
glove_Max-Len-Seq-20_WoRul_Freq-200_NB
glove_Max-Len-Seq-20_WoRul_Freq-200_RF
glove_Max-Len-Seq-20_WoRul_Freq-200_SVM
+++++++++++++++++

In [32]:
pathToClfModel = "" 
path2word2vecModel = 'dummyPath/word2vecModels/word2vec'
pathToWord2VecPretrained = "../word2vec_preTrainedModel/"

path_to_nonPreprocessedDataset = "../datasets/500_random_sample.csv"
path_to_PreprocessedDataset = "../datasets/500_random_sample_processed.csv"
pathToVectFolder = "../simple_vect/"
pathToClfFolder = "../simple_clfModels/"
obj = UsePretrainedVectClf(pathToVectFolder, pathToClfModel, pathToWord2VecPretrained, path_to_PreprocessedDataset, \
                   path_to_nonPreprocessedDataset)
# obj.generatePredNGramsPretrained(path_to_nonPreprocessedDataset, path_to_PreprocessedDataset, pathToVectFolder,\
#                                  pathToClfModel, path2word2vecModel, pathToWord2VecPretrained)

# path_to_nonPreprocessedDataset = "../datasets/500_random_sample_withPredictions.csv"
# path_to_PreprocessedDataset = "../datasets/500_random_sample_processed.csv"
# pathToVectFolder = "../tfidf_vect/"
# pathToClfFolder = "../tfidf_clfModels//"
# obj = UsePretrainedVectClf(pathToVectFolder, pathToClfModel, pathToWord2VecPretrained, path_to_PreprocessedDataset, \
#                    path_to_nonPreprocessedDataset)
# obj.generatePredNGramsPretrained(path_to_nonPreprocessedDataset, path_to_PreprocessedDataset, pathToVectFolder,\
#                                  pathToClfModel, path2word2vecModel, pathToWord2VecPretrained)

++++++++++++++++++++++++++++++++++++++++
Bigrams_WRul_CountVect_Freq-2959_GB
Bigrams_WRul_CountVect_Freq-2959_LR
Bigrams_WRul_CountVect_Freq-2959_MLP
Bigrams_WRul_CountVect_Freq-2959_NB
Bigrams_WRul_CountVect_Freq-2959_RF
Bigrams_WRul_CountVect_Freq-2959_SVM
Bigrams_WRul_CountVect_Freq-2959_DT
++++++++++++++++++++++++++++++++++++++++
BiTrigrams_WRul_CountVect_Freq-6502_DT
BiTrigrams_WRul_CountVect_Freq-6502_GB
BiTrigrams_WRul_CountVect_Freq-6502_LR
BiTrigrams_WRul_CountVect_Freq-6502_MLP
BiTrigrams_WRul_CountVect_Freq-6502_NB
BiTrigrams_WRul_CountVect_Freq-6502_RF
BiTrigrams_WRul_CountVect_Freq-6502_SVM
++++++++++++++++++++++++++++++++++++++++
Trigrams_WRul_CountVect_Freq-3561_DT
Trigrams_WRul_CountVect_Freq-3561_GB
Trigrams_WRul_CountVect_Freq-3561_LR
Trigrams_WRul_CountVect_Freq-3561_MLP
Trigrams_WRul_CountVect_Freq-3561_NB
Trigrams_WRul_CountVect_Freq-3561_RF
Trigrams_WRul_CountVect_Freq-3561_SVM
++++++++++++++++++++++++++++++++++++++++
UniAndBigrams_WRul_CountVect_Freq-4215_GB
UniA

In [ ]:
pathToClfModel = "" 
path_to_nonPreprocessedDataset = "../datasets/500_random_sample.csv"
path_to_PreprocessedDataset = "../datasets/500_random_sample_processed.csv"
pathToVectFolder = "../simple_vect/"
pathToClfFolder = "../simple_clfModels/"
obj = UsePretrainedVectClf(pathToVectFolder, pathToClfModel, path_to_PreprocessedDataset, path_to_nonPreprocessedDataset, include_rules = False)


In [ ]:
class Use_Word2Vec:
    def __init__(self,pathToTokenizer, pathToClfModel, pathToWord2VecPretrained, path_to_PreprocessedDataset, \
                 path_to_nonPreprocessedDataset, include_rules = False):
        self.pathToTokenizer = pathToTokenizer
        self.pathToClfModel = pathToClfModel
        self.include_rules = include_rules
        self.pathToWord2VecPretrained = pathToWord2VecPretrained
        self.path_to_nonPreprocessedDataset = path_to_nonPreprocessedDataset
        self.path_to_PreprocessedDataset = path_to_PreprocessedDataset

    


    
 
        

        

    

pathToClfModel = "" 
path_to_nonPreprocessedDataset = "../datasets/500_random_sample.csv"
path_to_PreprocessedDataset = "../datasets/500_random_sample_processed.csv"
pathToVectFolder = "../word2vec_vect/"
pathToClfFolder = "../word2vec_clfModels/"


path2word2vecModel = 'dummyPath/word2vecModels/word2vec'
pathToWord2VecPretrained = "../word2vec_preTrainedModel/"
obj = Use_Word2Vec(pathToVectFolder, pathToClfModel, pathToWord2VecPretrained, path_to_PreprocessedDataset, \
                   path_to_nonPreprocessedDataset)
obj.generatePredWord2VecPretrained(path_to_nonPreprocessedDataset, path_to_PreprocessedDataset, pathToVectFolder,\
                                 pathToClfModel, path2word2vecModel, pathToWord2VecPretrained)

In [19]:
class Use_Word2Vec:
    def __init__(self,pathToTokenizer, pathToClfModel, pathToWord2VecPretrained, path_to_PreprocessedDataset, \
                 path_to_nonPreprocessedDataset, include_rules = False):
        self.pathToTokenizer = pathToTokenizer
        self.pathToClfModel = pathToClfModel
        self.include_rules = include_rules
        self.pathToWord2VecPretrained = pathToWord2VecPretrained
        self.path_to_nonPreprocessedDataset = path_to_nonPreprocessedDataset
        self.path_to_PreprocessedDataset = path_to_PreprocessedDataset    def __init__(self,pathToTokenizer, pathToClfModel, pathToWord2VecPretrained, path_to_PreprocessedDataset, \
                 path_to_nonPreprocessedDataset, include_rules = False):
        self.pathToTokenizer = pathToTokenizer
        self.pathToClfModel = pathToClfModel
        self.include_rules = include_rules
        self.pathToWord2VecPretrained = pathToWord2VecPretrained
        self.path_to_nonPreprocessedDataset = path_to_nonPreprocessedDataset
    
    def word_vector_avg(self, inv_tokenizer, fun_word_to_vec_map, tw_sequence, size):
        vec = np.zeros(size)
        count = 0
        columnNameList = []
        for i in range(18):
            columnName = "rule" + str(i + 1)
            columnNameList.append(columnName)
        for seq in tw_sequence:
            try:
                if inv_tokenizer[seq] in columnNameList:
                    embedding_vector = np.ndarray(shape=(size,))
                    embedding_vector[:] = 1/size
                else:
                    embedding_vector = fun_word_to_vec_map[inv_tokenizer[seq]]
                if embedding_vector is not None:
                    vec += embedding_vector
                else:
                    vec += np.zeros(size)
                count += 1.
            except KeyError:  # handling the case where the token is not in vocabulary
                continue
        if count != 0:
            vec /= count
        return vec

    
    def concat_sparse_matrices_h(self, data_X_dtm, data_Rules_dtm, features_X, features_Rules):
        combined_features = features_X + features_Rules
        concat_sparse = scipy.sparse.hstack([data_X_dtm, data_Rules_dtm], format='csr')
        return concat_sparse, combined_features
    
    def series_to_DataFrame(self, X_data):
        X_data = X_data.to_frame()
        return X_data
    
    def apply_regex_word2vec(self, text, regex, ruleTokenizerSequenceIndex):
        match_found = (re.search(regex, text) != None)
        if match_found:
            match_found = int(ruleTokenizerSequenceIndex)
        else:
            match_found = 0
        return match_found
    
    def gen_rules_features_word2vec(self, X_data_series, tokenizer):  # , X_data_dtm, features_arg):
        '''sparse matrix and series matrices should be converted to dataframe for applying rules and treating
    it as features...
    I wrote two functions i.e.,sparse_matrix_to_DataFrame() and series_DataFrame()
      for changing datatypes'''
        X_data_DF = self.series_to_DataFrame(X_data_series)
        regexes = [
            re.compile(r'\b(I|we)\b.*\b(am|are|will be)\b.*\    
    def word_vector_avg(self, inv_tokenizer, fun_word_to_vec_map, tw_sequence, size):
        vec = np.zeros(size)
        count = 0
        columnNameList = []    
    def word_vector_avg(self, inv_tokenizer, fun_word_to_vec_map, tw_sequence, size):
        vec = np.zeros(size)
        count = 0
        columnNameList = []
        for i in range(18):
            columnName = "rule" + str(i + 1)
            columnNameList.append(columnName)
        for seq in tw_sequence:
            try:
                if inv_tokenizer[seq] in columnNameList:
                    embedding_vector = np.ndarray(shape=(size,))
                    embedding_vector[:] = 1/size
                else:
                    embedding_vector = fun_word_to_vec_map[inv_tokenizer[seq]]
                if embedding_vector is not None:
                    vec += embedding_vector
                else:
                    vec += np.zeros(size)
                count += 1.
            except KeyError:  # handling the case where the token is not in vocabulary
                continue
        if count != 0:
            vec /= count
        return vec

    
    def concat_sparse_matrices_h(self, data_X_dtm, data_Rules_dtm, features_X, features_Rules):
        combined_features = features_X + features_Rules
        concat_sparse = scipy.sparse.hstack([data_X_dtm, data_Rules_dtm], format='csr')
        return concat_sparse, combined_features
    
    def series_to_DataFrame(self, X_data):
        X_data = X_data.to_frame()
        return X_data
    
    def apply_regex_word2vec(self, text, regex, ruleTokenizerSequenceIndex):
        match_found = (re.search(regex, text) != None)
        if match_found:
            match_found = int(ruleTokenizerSequenceIndex)
        else:
            match_found = 0
        return match_found
    
    def gen_rules_features_word2vec(self, X_data_series, tokenizer):  # , X_data_dtm, features_arg):
        '''sparse matrix and series matrices should be converted to dataframe for applying rules and treating
    it as features...
    I wrote two functions i.e.,sparse_matrix_to_DataFrame() and series_DataFrame()
      for changing datatypes'''
        X_data_DF = self.series_to_DataFrame(X_data_series)
        regexes = [
            re.compile(r'\b(I|we)\b.*\b(am|are|will be)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b',
                       re.I | re.M),
            re.compile(r'\b(I\'m|Im)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
            re.compile(r'\b(we\'re|we are)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(bring|give|help|raise|donate|auction)\b',
                       re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(we\'ll|we will)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(ready|prepared)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
            re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(will be)\b.*\b(brought|given|raised|donated|auctioned)\b', re.I | re.M),
            re.compile(r'\b\w*\s*\b\?', re.I | re.M),
            re.compile(r'\b(you|u).*(can|could|should|want to)\b', re.I | re.M),
            re.compile(r'\b(can|could|should).*(you|u)\b', re.I | re.M),
            re.compile(r'\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M)

        ]
        temp = pd.DataFrame()
        features_arg = []
        for i, regex in zip(range(len(regexes)), regexes):
            # we can also use ruleString()
            columnName = "rule" + str(i + 1)
            features_arg.append(columnName)
            temp[columnName] = X_data_DF['tweet_text'].apply(lambda text: self.apply_regex_word2vec(text, regex, tokenizer.word_index[columnName]))

        temp_sparse = scipy.sparse.csr_matrix(temp.values)

        return temp_sparse, features_arg
    def apply_gen_rules_features_word2vec(self, X, X_train_WoR_dtm, tokenizer, X_train_WoR_Features):
        X_Rules_dtm, features_Rules = self.gen_rules_features_word2vec(X, tokenizer)

        X_train_WR_dtm, combined_features = self.concat_sparse_matrices_h(X_train_WoR_dtm, X_Rules_dtm,
                                                                          X_train_WoR_Features, features_Rules)
        return X_train_WR_dtm, combined_feature

        for i in range(18):
            columnName = "rule" + str(i + 1)
            columnNameList.append(columnName)
        for seq in tw_sequence:
            try:
                if inv_tokenizer[seq] in columnNameList:
                    embedding_vector = np.ndarray(shape=(size,))
                    embedding_vector[:] = 1/size
                else:
                    embedding_vector = fun_word_to_vec_map[inv_tokenizer[seq]]
                if embedding_vector is not None:
                    vec += embedding_vector
                else:
                    vec += np.zeros(size)
                count += 1.
            except KeyError:  # handling the case where the token is not in vocabulary
                continue
        if count != 0:
            vec /= count
        return vec

    
    def concat_sparse_matrices_h(self, data_X_dtm, data_Rules_dtm, features_X, features_Rules):
        combined_features = features_X + features_Rules
        concat_sparse = scipy.sparse.hstack([data_X_dtm, data_Rules_dtm], format='csr')
        return concat_sparse, combined_features
    
    def series_to_DataFrame(self, X_data):
        X_data = X_data.to_frame()
        return X_data
    
    def apply_regex_word2vec(self, text, regex, ruleTokenizerSequenceIndex):
        match_found = (re.search(regex, text) != None)
        if match_found:
            match_found = int(ruleTokenizerSequenceIndex)
        else:
            match_found = 0
        return match_found
    
    def gen_rules_features_word2vec(self, X_data_series, tokenizer):  # , X_data_dtm, features_arg):
        '''sparse matrix and series matrices should be converted to dataframe for applying rules and treating
    it as features...
    I wrote two functions i.e.,sparse_matrix_to_DataFrame() and series_DataFrame()
      for changing datatypes'''
        X_data_DF = self.series_to_DataFrame(X_data_series)
        regexes = [
            re.compile(r'\b(I|we)\b.*\b(am|are|will be)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b',
                       re.I | re.M),
            re.compile(r'\b(I\'m|Im)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
            re.compile(r'\b(we\'re|we are)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(bring|give|help|raise|donate|auction)\b',
                       re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(we\'ll|we will)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(ready|prepared)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
            re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(will be)\b.*\b(brought|given|raised|donated|auctioned)\b', re.I | re.M),
            re.compile(r'\b\w*\s*\b\?', re.I | re.M),
            re.compile(r'\b(you|u).*(can|could|should|want to)\b', re.I | re.M),
            re.compile(r'\b(can|could|should).*(you|u)\b', re.I | re.M),
            re.compile(r'\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M)

        ]
        temp = pd.DataFrame()
        features_arg = []
        for i, regex in zip(range(len(regexes)), regexes):
            # we can also use ruleString()
            columnName = "rule" + str(i + 1)
            features_arg.append(columnName)
            temp[columnName] = X_data_DF['tweet_text'].apply(lambda text: self.apply_regex_word2vec(text, regex, tokenizer.word_index[columnName]))

        temp_sparse = scipy.sparse.csr_matrix(temp.values)

        return temp_sparse, features_arg
    def apply_gen_rules_features_word2vec(self, X, X_train_WoR_dtm, tokenizer, X_train_WoR_Features):
        X_Rules_dtm, features_Rules = self.gen_rules_features_word2vec(X, tokenizer)

        X_train_WR_dtm, combined_features = self.concat_sparse_matrices_h(X_train_WoR_dtm, X_Rules_dtm,
                                                                          X_train_WoR_Features, features_Rules)
        return X_train_WR_dtm, combined_feature
b(bringing|giving|helping|raising|donating|auctioning)\b',
                       re.I | re.M),
            re.compile(r'\b(I\'m|Im)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
            re.compile(r'\b(we\'re|we are)\b.*\b(bringing|giving|helping|raising|donating|auctioning)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(bring|give|help|raise|donate|auction)\b',
                       re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(will|would like to)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(we\'ll|we will)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(ready|prepared)\b.*\b(bring|give|help|raise|donate|auction)\b', re.I | re.M),
            re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(where)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(I|we)\b.*\b(like|want)\b.*\bto\b.*\b(work|volunteer|assist)\b', re.I | re.M),
            re.compile(r'\b(will be)\b.*\b(brought|given|raised|donated|auctioned)\b', re.I | re.M),
            re.compile(r'\b\w*\s*\b\?', re.I | re.M),
            re.compile(r'\b(you|u).*(can|could|should|want to)\b', re.I | re.M),
            re.compile(r'\b(can|could|should).*(you|u)\b', re.I | re.M),
            re.compile(r'\b(like|want)\b.*\bto\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(bring|give|help|raise|donate)\b', re.I | re.M),
            re.compile(r'\b(how)\b.*\b(can I|can we)\b.*\b(work|volunteer|assist)\b', re.I | re.M)

        ]
        temp = pd.DataFrame()
        features_arg = []
        for i, regex in zip(range(len(regexes)), regexes):
            # we can also use ruleString()
            columnName = "rule" + str(i + 1)
            features_arg.append(columnName)
            temp[columnName] = X_data_DF['tweet_text'].apply(lambda text: self.apply_regex_word2vec(text, regex, tokenizer.word_index[columnName]))

        temp_sparse = scipy.sparse.csr_matrix(temp.values)

        return temp_sparse, features_arg
    def apply_gen_rules_features_word2vec(self, X, X_train_WoR_dtm, tokenizer, X_train_WoR_Features):
        X_Rules_dtm, features_Rules = self.gen_rules_features_word2vec(X, tokenizer)

        X_train_WR_dtm, combined_features = self.concat_sparse_matrices_h(X_train_WoR_dtm, X_Rules_dtm,
                                                                          X_train_WoR_Features, features_Rules)
        return X_train_WR_dtm, combined_feature

    
    def traditional_ChangeVectPath(self, pathToTokenizer):
        self.pathToTokenizer = pathToTokenizer 
        
    def word2vec_ChangeWord2VecPretrainedPath(self, pathToWord2VecPretrained):
        self.pathToWord2VecPretrained = pathToWord2VecPretrained 

    def word2vec_loadWord2VecPretrained(self):  
        word2vec_model = pickle.load(open(self.pathToWord2VecPretrained, 'rb'))
        return word2vec_model
        
    def traditional_loadVect(self):
        Vect = pickle.load(open(self.pathToTokenizer, 'rb'))
        return Vect
    
    def traditional_loadClf(self):  
        clf_model = pickle.load(open(self.pathToClfModel, 'rb'))
        return clf_model

    def traditional_ChangeClfModelPath(self, pathToClfModel):
        self.pathToClfModel = pathToClfModel  

    def invTokenizer(self, tokenizer):
        inv_tokenizer = {v: k for k, v in tokenizer.word_index.items()}
        return inv_tokenizer

    def word2vec_Vect(self, pathToWord2VecPretrained, df):
        tokenizer = pickle.load(open(self.pathToTokenizer, 'rb'))
        embed_dim = int(self.pathToTokenizer.split("Freq-")[1])
        if "WRul" in self.pathToTokenizer:
            MAX_SEQUENCE_LENGTH = int(self.pathToTokenizer.split("Seq-")[1].split("_")[0]) - 18
        else:
            MAX_SEQUENCE_LENGTH = int(self.pathToTokenizer.split("Seq-")[1].split("_")[0])

        sequences = tokenizer.texts_to_sequences(df['tweet_text'])
        X_train_WoR_dtm = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
        X_train_Features = ["_Not_Apply_"]
        # print(type(X_train_WoR_dtm), X_train_WoR_dtm.shape)
        if self.include_rules:
            df_notPreprocessed = pd.read_csv(self.path_to_nonPreprocessedDataset)
            X_train_dtm, _ = self.apply_gen_rules_features(df_notPreprocessed["tweet_text"], X_train_WoR_dtm, tokenizer, \
                                                      X_train_Features)
        else:
            X_train_dtm = X_train_WoR_dtm
        inv_tokenizer = self.invTokenizer(tokenizer)
        fun_word_to_vec_map = self.load_word2vec_model(pathToWord2VecPretrained)
        vector = []
        for tw_sequence in X_train_dtm:
            vec = self.word_vector_avg(inv_tokenizer, fun_word_to_vec_map, tw_sequence, embed_dim)
            vector.append(vec)
        X_train_dtm = np.array(vector)
        return X_train_dtm
    
    def load_word2vec_model(self, path):
        if "Model" not in path:
            print("You may not loading a model; You may loading a tokenizer.")
        if "glove".lower() in path.lower().split("/")[-1]:
            word_to_vec_map = pickle.load(open(path, 'rb'))
        elif "word2vec".lower() in path.lower().split("/")[-1]:
            word_to_vec_map = pickle.load(open(path, 'rb'))
        elif "google".lower() in path.lower().split("/")[-1]:
            word_to_vec_map = pickle.load(open(path, 'rb'))
        elif "doc2vec".lower() in path.lower().split("/")[-1]:
            word_to_vec_map = pickle.load(open(path, 'rb'))
            
        elif "crisisNLP2vec".lower() in path.lower().split("/")[2]:
             word_to_vec_map = pickle.load(open(path, 'rb'))
        return word_to_vec_map
    
    def get_predictions(self, clf_model, X):
        return clf_model.predict(X)
    
    def generateNameFromDataSetName(self, list1):
        str1 = ""
        for e in list1:
            if e == "":
                str1 += '.'.join(str(e))
            else:
                if e == "csv":
                    e = "_withPredictions." + e
                    str1 += ''.join(str(e))
                else:
                    str1 += ''.join(str(e))
        return ".." + str1

    def generatePredWord2VecPretrained(self, path_to_nonPreprocessedDataset, path_to_PreprocessedDataset, pathToVectFolder,\
                                     pathToClfModel, path2word2vecModel, pathToWord2VecPretrained):
        NotProcessedDataWithPredictions = pd.read_csv(obj.path_to_nonPreprocessedDataset, encoding = "ISO-8859-1")
        df = pd.read_csv(obj.path_to_PreprocessedDataset,  encoding = "ISO-8859-1")
        for vect_filename in glob.glob(pathToVectFolder + "/*"):
            print(40*"+")
            obj.traditional_ChangeVectPath(vect_filename)
            Vect = obj.traditional_loadVect()
            for pretrainedWord2VecModel_filename in glob.glob(pathToWord2VecPretrained + "*"): 
                if pretrainedWord2VecModel_filename.split("/")[-1].replace("Model", "Vect").split("_") == \
                vect_filename.split("/")[-1].split("_"):
                    obj.word2vec_ChangeWord2VecPretrainedPath(pretrainedWord2VecModel_filename)
                    for clfModel_filename in glob.glob(pathToClfFolder + "*"):
                        if clfModel_filename.split("/")[-1].split("_")[6:-1] == pretrainedWord2VecModel_filename.split("/")[-1].replace("Model", "Vect").split("_"):
                            obj.traditional_ChangeClfModelPath(clfModel_filename)
                            name_cell = obj.pathToClfModel.split("/")[-1].split("_")[5:]
                            if name_cell[-3] == "WRul":
                                obj.include_rules = True
                            else:
                                obj.include_rules = False
                            X = obj.word2vec_Vect(pretrainedWord2VecModel_filename, df)
                            clf_model = obj.traditional_loadClf()
                            y_pred = obj.get_predictions(clf_model, X)
                            name_cell = name_cell[-6] + "_" + name_cell[-3]  + "_" + name_cell[-4] + "_" + name_cell[-2] + "_" + name_cell[-1]
                            print(name_cell)
                            NotProcessedDataWithPredictions[name_cell] = y_pred.tolist()
        notProcessedDatasetName = obj.generateNameFromDataSetName(obj.path_to_nonPreprocessedDataset.split("."))
        NotProcessedDataWithPredictions.to_csv(notProcessedDatasetName)
        print(40*"-")
    

pathToClfModel = "" 
path_to_nonPreprocessedDataset = "../datasets/500_random_sample.csv"
path_to_PreprocessedDataset = "../datasets/500_random_sample_processed.csv"
pathToVectFolder = "../word2vec_vect/"
pathToClfFolder = "../word2vec_clfModels/"


path2word2vecModel = 'dummyPath/word2vecModels/word2vec'
pathToWord2VecPretrained = "../word2vec_preTrainedModel/"
obj = Use_Word2Vec(pathToVectFolder, pathToClfModel, pathToWord2VecPretrained, path_to_PreprocessedDataset, \
                   path_to_nonPreprocessedDataset)
obj.generatePredWord2VecPretrained(path_to_nonPreprocessedDataset, path_to_PreprocessedDataset, pathToVectFolder,\
                                 pathToClfModel, path2word2vecModel, pathToWord2VecPretrained)

SyntaxError: EOL while scanning string literal (1119743166.py, line 61)

In [40]:
pretrainedWord2VecModel_filename.split("/")[-1]

'22-05-13_17-07-57_word2vec_Model_WRul_Max-Len-Seq-38_Freq-300'

In [ ]:
../word2vec_clfModels/_DT ../word2vec_vect/22-05-06_22-56-41_word2vec_Model_WoRul_Max-Len-Seq-26_Freq-300

In [ ]:
[22-05-06_22-56-41_Vect_WoRul_Max-Len-Seq-26_Freq-300_22-05-06_22-56-41_word2vec_Vect_WoRul_Max-Len-Seq-26_Freq-300]
==
['22-05-06', '22-56-41', 'word2vec', 'Model', 'WoRul', 'Max-Len-Seq-26', 'Freq-300']